In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/diegofiori/Desktop/epfl/master_thesis/master_thesis/')
from utils import read_pickle
from tqdm.notebook import tqdm

In [2]:
path = '/Users/diegofiori/Desktop/epfl/master_thesis/results/'
sim_code = 'cs'
density = read_pickle(path+f'physical_features_{sim_code}.pickle')
slices = read_pickle(path+f'slices_top_features_{sim_code}.pickle')

In [3]:
density = np.concatenate(density)

In [4]:
density.shape

(1888,)

In [5]:
slices.shape

(3, 150720, 34)

In [6]:
slices = [slices[:, i:i+80] for i in range(0, slices.shape[1], 80)]

In [7]:
slices = np.concatenate([np.expand_dims(slice_, axis=1) for slice_ in slices], axis=1)

In [8]:
slices.shape

(3, 1884, 80, 34)

In [9]:
indexes = []
arrays = []
for i in range(slices.shape[0]):
    for j in range(0, slices.shape[2]):
        for k in range(slices.shape[3]):
            arrays.append(slices[i, :, j, k].reshape((-1, 1)))
            indexes.append((i, j, k))
df = pd.DataFrame(data=np.concatenate(arrays, axis=1), columns=indexes)

In [10]:
df.shape

(1884, 8160)

In [11]:
df.head()

,"(0, 0, 0)","(0, 0, 1)","(0, 0, 2)","(0, 0, 3)","(0, 0, 4)","(0, 0, 5)","(0, 0, 6)","(0, 0, 7)","(0, 0, 8)","(0, 0, 9)",...,"(2, 79, 24)","(2, 79, 25)","(2, 79, 26)","(2, 79, 27)","(2, 79, 28)","(2, 79, 29)","(2, 79, 30)","(2, 79, 31)","(2, 79, 32)","(2, 79, 33)"
0,0.015552,0.013046,0.218468,0.218468,0.218710,0.218716,0.218468,0.218468,0.023863,0.023863,...,0.320736,0.320736,0.122949,0.122949,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,0.002504,0.000757,0.000004,0.000004,0.007596,0.007634,0.000096,0.000097,0.000000,0.000000,...,0.691308,0.691308,0.227454,0.227454,2.776021e-06,2.797134e-06,0.000069,0.000275,0.000009,0.000021
2,8.843659,9.419680,1.000000,1.000000,1.000148,1.000155,1.000000,1.000000,0.499949,0.499949,...,0.228067,0.228067,0.115696,0.115696,5.559640e-07,5.559475e-07,0.000006,0.000014,0.000002,0.000003
3,0.004740,0.002396,0.218465,0.218465,0.218658,0.218657,0.218465,0.218465,0.023857,0.023857,...,0.160403,0.160403,0.052572,0.052572,5.575423e-07,5.573634e-07,0.000004,0.000017,0.000001,0.000003
4,0.001981,0.000301,0.000004,0.000004,0.019037,0.019002,0.000182,0.000182,0.000000,0.000000,...,1.368144,1.368144,0.423149,0.423149,5.576061e-07,5.571803e-07,0.000007,0.000027,0.000002,0.000004


## Selecting features based on RandomForest

We firstly defined the regression problem.

In [12]:
import h5py

In [13]:
from scipy.io import loadmat
is_before = False
if is_before:
    x1 = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_{sim_code}1.mat')['k_quantities']
    x2 = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_{sim_code}2.mat')['k_quantities']
    y = np.concatenate([x1, x2])
else:
    x = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_{sim_code}.mat')
    y = x['k_quantities']

In [14]:
y = y[:len(df)]

In [15]:
non_zero_ind = np.any(y>0, axis=1)

In [16]:
y1 = y[non_zero_ind, 0]
y2 = y[non_zero_ind, 1]

In [17]:
X = df.values[non_zero_ind]

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [19]:
X = np.nan_to_num(X)
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.2)

In [20]:
model1 = RandomForestRegressor(n_estimators=100)
model1.fit(X_train, y1_train)
model2 = RandomForestRegressor(n_estimators=100)
model2.fit(X_train, y2_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [21]:
score_1 = model1.score(X_test, y1_test)
score_2 = model2.score(X_test, y2_test)
total_score = score_1 + score_2

In [22]:
score_2

0.8222624941009837

In [23]:
total_score

1.3137104141552607

In [24]:
model1.predict(X_test[19:20])

array([0.03664259])

In [25]:
y1_test[19]

0.037950038039069114

In [26]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y1_test, model1.predict(X_test))

0.003690385719159677

In [27]:
print('relative error')
mean_absolute_error(y1_test, model1.predict(X_test))/np.mean(y1_test)

relative error


0.10030157474853706

In [28]:
feature_importance = score_1/total_score*model1.feature_importances_ + score_2/total_score*model2.feature_importances_


In [29]:
# group the features (we want to keep all the spatial dimensions)
n_features = 34
feature_importance_temp = [feature_importance[i:i+n_features] for i in range(0, len(feature_importance), n_features)]
feature_importance_temp = [np.mean(feature_importance_temp[i:i+80], axis=0) 
                           for i in range(0, len(feature_importance_temp), 80)]

In [30]:
feature_importance_idx = [(i, j) for i in range(len(feature_importance_temp))
                          for j in range(n_features)]
feature_importance_idx = sorted(feature_importance_idx, reverse=True,
                                key=lambda x: feature_importance_temp[x[0]][x[1]])

In [48]:
from scipy.stats import pearsonr
def compute_correlation_with_old_features(target_feat, old_features_idx, X):
    correlations = []
    for old_idx in old_features_idx:
        old_feat = X[:, old_idx[0]*80*n_features: (old_idx[0]+1)*80*n_features][:, old_idx[1]::n_features]
        corr = [pearsonr(target_feat[:, i], old_feat[:, j])[0] for i in range(target_feat.shape[1]) 
                for j in range(old_feat.shape[1])]
        correlations.append(np.min(np.abs(corr)))
    return np.array(correlations)

In [49]:
important_features = []
threshold = 0.85
minimum_value = 1e-8
for idx in tqdm(feature_importance_idx):
    feat_value = feature_importance_temp[idx[0]][idx[1]]
    if feat_value < minimum_value:
        break
    feat_mat = X[:, idx[0]*80*n_features: (idx[0]+1)*80*n_features][:, idx[1]::n_features]
    if important_features:
        corrs = [compute_correlation_with_old_features(feat_mat, important_features, X)]
        if np.max(corrs) < threshold:
            important_features.append(idx)
    else:
        important_features.append(idx)
    

In [57]:
important_features_idx = important_features

In [58]:
important_features_idx

[(2, 31),
 (1, 1),
 (1, 16),
 (2, 33),
 (0, 31),
 (2, 1),
 (2, 0),
 (0, 17),
 (2, 29),
 (1, 29),
 (0, 29),
 (0, 28),
 (1, 28),
 (2, 28),
 (1, 32),
 (2, 18),
 (0, 32),
 (2, 32),
 (0, 23),
 (2, 25),
 (0, 18),
 (0, 21),
 (0, 20),
 (1, 20),
 (0, 24),
 (0, 22),
 (0, 25),
 (1, 22),
 (0, 14),
 (0, 27),
 (1, 21),
 (2, 26)]

In [59]:
len(important_features_idx)

32

In [60]:
from utils import write_pickle
write_pickle(path=path+f'selected_index_{sim_code}.pickle', array=important_features_idx)